# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [10]:
# import libraries

import nltk
import numpy as np
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import pandas as pd
from sqlalchemy import create_engine
import re
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.tree import DecisionTreeClassifier
import pickle
from sqlalchemy import create_engine
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.svm import SVC
import warnings
warnings.simplefilter('ignore')
import re
import pickle




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
# load data from database

engine = create_engine('sqlite:///HUMANIST80.db')
df = pd.read_sql("SELECT * FROM H_PERSONX", engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)
category_names = Y.columns.tolist()

### 2. Write a tokenization function to process your text data

In [12]:
def tokenize(text):

    # Converting everything to lower case
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize words
    tokens = word_tokenize(text)
    
    # normalization word tokens and remove stop words
    normlizer = PorterStemmer()
    stop_words = stopwords.words("english")
    
    normlized = [normlizer.stem(word) for word in tokens if word not in stop_words]
    
    return normlized

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
RFC = MultiOutputClassifier(RandomForestClassifier(n_estimators = 10))

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', RFC)
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=42)

In [15]:
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
from sklearn.metrics import classification_report

Y_pred = pipeline.predict(X_test)

In [17]:
for i, col in enumerate(Y_test):
    print(col)
    print(classification_report(Y_test[col], Y_pred[:, i]))

related
             precision    recall  f1-score   support

          0       0.62      0.47      0.54      1931
          1       0.84      0.91      0.87      5934

avg / total       0.79      0.80      0.79      7865

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      6533
          1       0.78      0.43      0.56      1332

avg / total       0.88      0.88      0.87      7865

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7829
          1       0.00      0.00      0.00        36

avg / total       0.99      1.00      0.99      7865

aid_related
             precision    recall  f1-score   support

          0       0.75      0.85      0.80      4646
          1       0.73      0.60      0.66      3219

avg / total       0.75      0.75      0.74      7865

medical_help
             precision    recall  f1-score   support

          0       0.93      1.00      0

### 6. Improve your model
Use grid search to find better parameters. 

In [18]:
def performance_metric(y_true, y_pred):
    """Calculate median F1 score for all of the output classifiers
    
    Args:
    y_true: array. Array containing actual labels.
    y_pred: array. Array containing predicted labels.
        
    Returns:
    score: float. Median F1 score for all of the output classifiers
    """
    f1_list = []
    for i in range(np.shape(y_pred)[1]):
        f1 = f1_score(np.array(y_true)[:, i], y_pred[:, i])
        f1_list.append(f1)
        
    score = np.median(f1_list)
    return score

In [19]:
parameters = {
    'tfidf__ngram_range': ((1, 1), (1, 2)),
    'tfidf__max_df': (0.8, 1.0),
    'tfidf__max_features': (None, 10000),
    'forest__estimator__n_estimators': [50, 100],
    'forest__estimator__min_samples_split': [2, 4]
}


cv = GridSearchCV(pipeline, parameters, cv=3, n_jobs=-1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
def evaluate_model(model, X_test, Y_test):
    Y_pred = model.predict(X_test)
    print(classification_report(Y_test, Y_pred, target_names = category_names))

In [24]:
print('Acc. : ', accuracy_score(Y_test, Y_pred))
print('Prec. : ', precision_score(Y_test, Y_pred, average='weighted'))
print('Recall: ', recall_score(Y_test, Y_pred, average='weighted'))


Acc. :  0.247679593134
Prec. :  0.726064196371
Recall:  0.478881209329


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [27]:
# Get results and add them to a dataframe.
def get_results(y_test, y_pred):
    results = pd.DataFrame(columns=['Category', 'f_score', 'precision', 'recall'])
    num = 0
    for cat in y_test.columns:
        precision, recall, f_score, support = precision_recall_fscore_support(y_test[cat], y_pred[:,num], average='weighted')
        results.set_value(num+1, 'Category', cat)
        results.set_value(num+1, 'f_score', f_score)
        results.set_value(num+1, 'precision', precision)
        results.set_value(num+1, 'recall', recall)
        num += 1
    print('Agg. f_score : ', results['f_score'].mean())
    print('Agg. precision : ', results['precision'].mean())
    print('Agg. recall : ', results['recall'].mean())
    return results

In [28]:
# testing a pure decision tree classifier
DTC = MultiOutputClassifier(DecisionTreeClassifier())

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', DTC)
    ])

X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train.as_matrix(), y_train.as_matrix())
y_pred = pipeline.predict(X_test)
results = get_results(y_test, y_pred)
results

Aggregated f_score: 0.932970139103
Aggregated precision: 0.931755943016
Aggregated recall: 0.934374258299


,Category,f_score,precision,recall
1,related,0.775906,0.773327,0.779219
2,request,0.855019,0.85389,0.856271
3,offer,0.992526,0.991767,0.993287
4,aid_related,0.710514,0.71022,0.710864
5,medical_help,0.898337,0.894287,0.903113
6,medical_products,0.944591,0.944418,0.944767
7,search_and_rescue,0.959867,0.959412,0.96033
8,security,0.966621,0.964726,0.968569
9,military,0.957152,0.955064,0.959719
10,child_alone,1,1,1


### 9. Export your model as a pickle file

In [32]:
import joblib
joblib.dump(results, 'disaster_model_hum80.pkl')

['disaster_model_hum80.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.